# MVSep-MDX23 Colab Fork v2.4
Adaptation de l'algorithme MVSep-MDX23 pour Colab, avec quelques ajustements :


[MVSEP-MDX23-Colab_v2.4_FR](https://colab.research.google.com/github/GiGiDKR/MVSEP-MDX23-Colab_v2/blob/v2.4/MVSep-MDX23-Colab-French.ipynb)

Changements récents :
<font size=2>

[**v2.4**](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/tree/v2.4)
* Modèles BS-Roformer de viperx ajoutés
* Modèle MDX-InstHQ4 ajouté en option
* Sortie FLAC
* Contrôler le gain du volume d'entrée
* Option filtrer les voix en dessous de 50Hz
* Meilleur algorithme de chunking (pas de clics)
* Un peu de nettoyage de code

</font>
<br>

<details>
    <summary>Journal des modifications complet :</summary>
<br>
<font size=2>
<br>

[**v2.3**](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/tree/v2.3)
* Modèle HQ3-Instr remplacé par VitLarge23 (grâce à MVSep)
* Traitement MDXv2 amélioré (grâce à Anjok)
* Algo BigShifts amélioré (v2)
* Traitement BigShifts ajouté à MDXv3 et VitLarge
* Traitement par lots de dossiers plus rapide

[**v2.2.2**](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/tree/v2.2)
* Amélioration du code de segmentation MDXv3 (grâce à HymnStudio)
* Modèle de démonstration D1581 remplacé par le nouveau modèle InstVocHQ MDXv3.
<br>

**v2.2.1**
* Ajout d'une fonctionnalité de poids personnalisés
* Correction de quelques bugs
* Entrée fixe : vous pouvez désormais utiliser un fichier ou un dossier comme entrée
<br>

**v2.2**
* Ajout de la compatibilité MDXv3
* Ajout du modèle de démonstration MDXv3 D1581 dans l'ensemble multibande de tige de chant.
* Ajout du SRS pleine bande VOC-FT au lieu de UVR-MDX-Instr-HQ3.
* Ajout de la fonctionnalité 2Pistes : sortie uniquement des voix / instruments (traitement plus rapide)
* Ajout d'une option de format de sortie 16 bits
* Ajout de "l'astuce BigShift" pour les modèles MDX
* Ajout de valeurs de chevauchement séparées pour MDX, MDXv3 et Demucs
* Réglage fin de la compensation de volume fixe pour MDX-VOC-FT
<br>

[**v2.1 (par deton24)**](https://github.com/deton24/MVSEP-MDX23-Colab_v2.1)
* Mis à jour avec MDX-VOC-FT au lieu de Kim Vocal 2
<br>

[**v2.0**](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/tree/2.0)
* Mis à jour avec les nouveaux modèles Kim Vocal 2 et UVR-MDX-Instr-HQ3
* Traitement par lots de dossiers
* Correction du saignement des hautes fréquences dans le chant
* Compensation de volume fixe pour les modèles MDX
<br>
</font>
</details>
<br>

Credits:
* [ZFTurbo/MVSep](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Modèles de [Demucs](https://github.com/facebookresearch/demucs), [Anjok](https://github.com/Anjok07/ultimatevocalremovergui) & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Adaptation et ajustements de [Jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)
* Traduction de [GiGiDKR](https://github.com/GiGiDKR/MVSEP-MDX23-Colab_v2.4_FR)
</font>

In [ ]:
#@markdown #Installation
#@markdown *Exécutez cette cellule pour installer MVSep-MDX23*
print('Installation... Cela prendra entre 1 et 15 minutes, selon les ressources disponibles de Colab...')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone -b v2.4 https://github.com/GiGiDKR/MVSEP-MDX23-Colab_v2 &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2.4_FR
print('Installation des dépendances...')
!pip install -r requirements.txt &> /dev/null
# correctif onnxruntime-gpu nightly pour cuda12.2
print('Installation du correctif onnxruntime CUDA 12.x...')
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/  &> /dev/null
print('Installation terminée !')

In [ ]:
#@markdown #Chargement de fichiers locaux
#@markdown *Exécutez cette cellule pour charger des fichiers audio (WAV, FLAC, MP3) ou une archive (ZIP) dans ce Colab.*

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Spécifier le répertoire de sortie, créer-le s'il n'existe pas
target_dir = '/content/audio_files/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Если файл - zip-архив
    if fn.endswith('.zip'):
        # Записать архив
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Décompresser l'archive
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Supprimer l'archive
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Archive "{name}" décompressée. Les fichiers sont dans : {folder}'.format(name=fn, folder=unzip_dir))

        # Afficher un bouton pour chaque fichier
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Fichier décompressé "{name}" avec une taille de {length} octets'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='Fichier enregistré : <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # Si le fichier n'est pas une archive zip
    # Enregistrer le fichier dans le répertoire
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='Fichier téléchargé par "{name}" avec une taille de {length} octets'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Vérifiez si le fichier est dans un format pris en charge : .wav, .mp3, .flac
    if not (fn.endswith('.wav') or fn.endswith('.mp3') or fn.endswith('.flac')):
        warning_text = widgets.HTML(
            value='<b style="color: red;">AVERTISSEMENT :</b>Vous téléchargez un fichier non pris en charge.'
        )
        display(warning_text)

    # Créer un lien cliquable vers le fichier
    file_path_text = widgets.HTML(
        value='Fichier enregistré : <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

### À propos des paramètres :


<font size=2>

* **BigShifts :** Meilleures performances qualité/vitesse avec des valeurs comprises entre 3 et 11, **MAIS** 11 ne donne pas toujours les meilleurs résultats. Pensez-y comme à une graine, des valeurs différentes donneront des résultats légèrement différents.<br>
Des valeurs plus élevées = un traitement plus long.
</font>

<font size=2>

* **Overlap InstVoc/VitLarge :** Aucun grand avantage à utiliser des valeurs élevées lorsque BigShifts est déjà élevé. Si vous utilisez BigShifts=1 (traitement régulier), vous pouvez utiliser des valeurs plus élevées comme 8 ou même 16.<br>
Des valeurs plus élevées = un traitement plus long.<br>
 *Il en va de même avec chevauchement_VOCFT, mais avec des valeurs comprises entre 0 et 0,95*
</font>

<font size=2>

* **Poids :** Quelle importance aura le résultat du modèle donné dans les résultats finaux.
</font>


In [ ]:
#@markdown #Separation
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2.4_FR

#@markdown ---
#@markdown #### Configuration de la séparation :
#@markdown Copier le chemin reçu dans la cellule de téléchargement si l'audio a été téléchargé localement !
#@markdown Généralement : /content/audio_files
input = '/content/drive/MyDrive/input' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}

output_format = 'FLOAT' #@param ["PCM_16", "FLOAT", "FLAC"]
Separation_mode = 'Vocals/Instrumental' #@param ["Vocals/Instrumental", "4-STEMS"]
input_gain = 0 #@param [0, -3, -6] {type:"raw"}
restore_gain_after_separation = False #@param {type:"boolean"}
filter_vocals_below_50hz = True #@param {type:"boolean"}
#@markdown ___
##@markdown

#@markdown  ### Configuration du modèle :

#@markdown *Définissez BigShifts=1 pour désactiver cette fonctionnalité*
BigShifts = 3 #@param {type:"slider", min:1, max:41, step:1}
#@markdown ---
BSRoformer_model = 'ep_317_1297' #@param ["ep_317_1297", "ep_368_1296"]
weight_BSRoformer = 10 #@param {type:"slider", min:0, max:10, step:1}
##@markdown ---
weight_InstVoc = 4 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_VitLarge = True #@param {type:"boolean"}
weight_VitLarge = 1 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_InstHQ4 = True #@param {type:"boolean"}
weight_InstHQ4 = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_InstHQ4 = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
use_VOCFT = True #@param {type:"boolean"}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_VOCFT = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
#@markdown *Demucs n'est utilisé qu'en mode 4-STEMS.*
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}

use_InstVoc_ = '--use_InstVoc' #forced use
use_BSRoformer_ =  '--use_BSRoformer' #forced use
use_VOCFT_ = '--use_VOCFT' if use_VOCFT is True else ''
use_VitLarge_ = '--use_VitLarge' if use_VitLarge is True else ''
use_InstHQ4_ = '--use_InstHQ4' if use_InstHQ4 is True else ''
restore_gain = '--restore_gain' if restore_gain_after_separation is True else ''
vocals_only = '--vocals_only' if Separation_mode == 'Vocals/Instrumental' else ''
filter_vocals = '--filter_vocals' if filter_vocals_below_50hz is True else ''

if Path(input).is_file():
  file_path = input
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --input_audio "{file_path}" \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_}


else:
    file_paths = sorted(glob.glob(input + "/*"))[:]
    input_audio_args = ' '.join([f'"{path}"' for path in file_paths])
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    !python inference.py \
        --input_audio {input_audio_args} \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_}